In [1]:
from dataset import SmokeDetectionDataset
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm



smkd_dataset = SmokeDetectionDataset('data/dataset/dataset_classification/test','data/dataset/dataset/wildfire_smoke_dataset/test/labels',sequence_length=1)
test_loader = DataLoader(dataset=smkd_dataset, batch_size=1, pin_memory=True)

2024-04-18 21:01:39.765422: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 21:01:39.792133: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 21:01:40.215234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


['nonsmoke', 'smoke']


load smoke images to cache: 100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


In [2]:
def dict_of_label(y):
    return [{
            'label': yy[0].item(),
            'box': [yy[1][0].item(), 
                    yy[1][1].item(), 
                    yy[1][2].item(), 
                    yy[1][3].item()],
            'conf': 1.0
            } for yy in y]


def iou(box1, box2):
        # Extract coordinates of the boxes
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2
    
    # Calculate the intersection coordinates
    x_left = max(xmin1, xmin2)
    y_top = max(ymin1, ymin2)
    x_right = min(xmax1, xmax2)
    y_bottom = min(ymax1, ymax2)
    
    # Calculate the intersection area
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    # Calculate the areas of the bounding boxes
    box1_area = (xmax1 - xmin1) * (ymax1 - ymin1)
    box2_area = (xmax2 - xmin2) * (ymax2 - ymin2)
    
    # Calculate the union area
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area if union_area > 0 else 0
    
    return iou

def calculate_detection_accuracy(predicted_boxes, gt_boxes, iou_threshold=0.3):
    """
    input: predicted_boxes, gt_boxes, iou_threshold
    return tp, tn, fp, fn
    """
    total_gt_objects = len(gt_boxes)
    total_predicted_objects = len(predicted_boxes)
    tp = tn = fp = fn = 0
    if total_gt_objects==0 & total_predicted_objects==0:
        tn = 1
        return tp, tn, fp, fn
    if total_gt_objects==0 & total_predicted_objects!=0:
        fp = total_predicted_objects
        return tp, tn, fp, fn
    if total_gt_objects!=0 & total_predicted_objects==0:
        fn = total_gt_objects

    match_matrix = np.zeros((total_predicted_objects,total_gt_objects))
    for pred_idx in range(total_predicted_objects):
        for gt_idx in range(total_gt_objects): 
            if iou(predicted_boxes[pred_idx],gt_boxes[gt_idx]) > iou_threshold:
                match_matrix[pred_idx][gt_idx] = 1
    gt_sum = np.sum(match_matrix, 0)
    tp = np.sum(gt_sum)
    fn = np.sum(gt_sum==0)
    fp = np.sum(np.sum(match_matrix,1)==0)
    return tp, tn, fp, fn



#  print(f'accuracy: {round(num_correct / num_samples, 4)}')

In [3]:
from model.yolov9.yolov9.detector import Detector

In [19]:
detector = Detector()

YOLOv5 🚀 1e33dbb Python-3.9.16 torch-2.2.2+cu121 CUDA:0 (Quadro RTX 4000, 7974MiB)



Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs


In [23]:
conf_threshold = 0.1
iou_threshold = 0.3
import time
# predicted_boxes
loop = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
total_tp = total_tn = total_fp = total_fn = 0
epsilon = 0.0000001
min_mm = 1
latency=[]
for x, y,images,ys in test_loader:
    
    x = np.array([np.transpose(i.numpy().squeeze(0), (2, 0, 1)) for i in x])
    if(len(y)!=0):
        y = dict_of_label(y)
    # else:
    #     continue
    start = time.time()
    preds = detector.detect(x,conf_threshold)
    # time,preds = detector.detect(x,conf_threshold)
    latency.append(time.time() - start)
    gt_boxes = list(map(lambda yy: yy['box'],y))
    # if len(y) == 0:
    #     continue
    
    pred_boxes = list(map(lambda yy: [yyy/640 for yyy in yy[:4].tolist()],[pred for pred in preds[0] if len(pred)!=0]))
    # print(gt_boxes)
    # print(preds)
    tp, tn, fp, fn = calculate_detection_accuracy(pred_boxes, gt_boxes,iou_threshold)
    # if fn !=0:
    #     print(f'frame: {images[0]}\npreds:{preds})\ntrust:{y}')
    total_tp+=tp
    total_tn+=tn
    total_fp+=fp
    total_fn+=fn
    # print(images[0],preds,y)
    # time.sleep(1)
    precision = total_tp/(total_tp+total_fp+epsilon)
    recall = total_tp/(total_tp+total_fn+epsilon)
    f1 = 2*precision*recall/(precision+recall+epsilon)
    acc = (total_tp+total_tn)/(total_tp+total_tn+total_fn+total_fp)
    loop.set_postfix(precision=f'{precision:.4f}', recall=f'{recall:.4f}', f1=f'{f1:.4f}', acc=f'{acc:.4f}')
    loop.update(1)

100%|█████████▉| 3036/3037 [00:40<00:00, 76.60it/s, acc=0.8363, f1=0.7207, precision=0.9014, recall=0.6004]

100%|██████████| 3037/3037 [00:52<00:00, 76.60it/s, acc=0.8363, f1=0.7207, precision=0.9014, recall=0.6004]

In [6]:
np.mean(latency)

0.01109890675599692

In [7]:
latency

[0.22630524635314941,
 0.01761150360107422,
 0.017399072647094727,
 0.01741337776184082,
 0.01741647720336914,
 0.017421722412109375,
 0.0173797607421875,
 0.017400741577148438,
 0.01749134063720703,
 0.017522096633911133,
 0.017551660537719727,
 0.016255855560302734,
 0.011279821395874023,
 0.01088404655456543,
 0.010751008987426758,
 0.010670185089111328,
 0.01055002212524414,
 0.010497093200683594,
 0.01052236557006836,
 0.010560035705566406,
 0.010494232177734375,
 0.010498523712158203,
 0.01053762435913086,
 0.01096034049987793,
 0.010528326034545898,
 0.01047658920288086,
 0.010456085205078125,
 0.010612964630126953,
 0.010616064071655273,
 0.010608673095703125,
 0.010574102401733398,
 0.01053166389465332,
 0.010929584503173828,
 0.010576009750366211,
 0.010560035705566406,
 0.010450601577758789,
 0.010460376739501953,
 0.01053309440612793,
 0.010463953018188477,
 0.01058816909790039,
 0.010614395141601562,
 0.010537862777709961,
 0.010543584823608398,
 0.010530710220336914,
 0.0

### Measure Object detector model latency 

In [8]:
import time
def timeit(fn): 
    # *args and **kwargs are to support positional and named arguments of fn
    def get_time(*args, **kwargs): 
        start = time.time() 
        output = fn(*args, **kwargs)
        print(f"Time taken in {fn.__name__}: {time.time() - start:.7f}")
        return output  # make sure that the decorator returns the output of fn
    return get_time 


In [9]:
detect_raw = detector.detector
# input = np.random.randint(0, 256, size=(1,3,640,640),dtype='int8')
input = np.random.randint(low=0, high=256, size=(1, 3, 640, 640), dtype=np.uint8)

AttributeError: 'Detector' object has no attribute 'detector'

100%|██████████| 3037/3037 [00:47<00:00, 79.81it/s, acc=0.8982, f1=0.8438, precision=0.9109, recall=0.7859]

In [ ]:
@timeit
def measure_object_detector_latency(input):
    for i in range(1000):
        detect_raw.detect_raw(input)

In [ ]:
# warm up
for i in range(500):
    detect_raw.detect_raw(input)
measure_object_detector_latency(input)
  


### Measure Motion cost

In [ ]:
from model.OurModel.object_detection import motion_measure
import numpy as np
import cv2
import time

In [ ]:
@timeit
def motion_measure_latency(input):
    for i in range(1000):
        motion_measure(input)
    

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(2)]
for i in range(1000):
    motion_measure_latency(image_list)

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(3)]
for i in range(1000):
    motion_measure_latency(image_list)

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(4)]
for i in range(1000):
    motion_measure_latency(image_list)

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(5)]
for i in range(1000):
    motion_measure_latency(image_list)